In [34]:
import yfinance as yf
import numpy as np
import pandas as pd

In [35]:
tickers = ["AMZN", "GOOG", "META"]
ohlcv_data = {}

In [36]:
for ticker in tickers:
    temp =  yf.download(ticker, period = '7mo', interval = '1d')
    temp.dropna(how = 'any', inplace = True)
    ohlcv_data[ticker] = temp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [37]:
def CAGR(DF):
    df = DF.copy()
    df["return"] = df["Adj Close"].pct_change()
    df["cum_return"] = (1 + df["return"]).cumprod()
    n = len(df)/252
    CAGR = (df["cum_return"][-1])**(1/n) - 1
    return CAGR

In [38]:
def volatility(DF):
    "function to calculate annualized volatility of a trading strategy"
    df = DF.copy()
    df["daily_ret"] = DF["Adj Close"].pct_change()
    vol = df["daily_ret"].std() * np.sqrt(252)
    return vol


In [39]:
def sharpe(DF, rf):
    "function to calculate Sharpe Ratio of a trading strategy"
    df = DF.copy()
    rd = 0.03
    return (CAGR(df) - rf)/volatility(df)

In [40]:
def sortino(DF, rf):
    "function to calculate Sortino Ratio of a trading strategy"
    df = DF.copy()
    df["return"] = df["Adj Close"].pct_change()
    neg_return = np.where(df["return"]>0,0,df["return"])
    #below you will see two ways to calculate the denominator (neg_vol), some people use the
    #standard deviation of negative returns while others use a downward deviation approach,
    #you can use either. However, downward deviation approach is more widely used
    neg_vol = np.sqrt((pd.Series(neg_return[neg_return != 0]) ** 2).mean() * 252)
    #neg_vol = pd.Series(neg_return[neg_return != 0]).std() * np.sqrt(252)
    return (CAGR(df) - rf)/neg_vol

In [41]:
for ticker in ohlcv_data:
    print("Sharpe of {} = {}".format(ticker,sharpe(ohlcv_data[ticker],0.03)))
    print("Sortino of {} = {}".format(ticker,sortino(ohlcv_data[ticker],0.03)))
    

Sharpe of AMZN = 2.424780562865726
Sortino of AMZN = 2.7395128552912387
Sharpe of GOOG = 3.363111413707972
Sortino of GOOG = 3.9071648762938094
Sharpe of META = 4.763570512553071
Sortino of META = 6.389721246318833


C:\Users\IVAN\AppData\Local\Temp\ipykernel_15332\667503414.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  CAGR = (df["cum_return"][-1])**(1/n) - 1
C:\Users\IVAN\AppData\Local\Temp\ipykernel_15332\667503414.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  CAGR = (df["cum_return"][-1])**(1/n) - 1
C:\Users\IVAN\AppData\Local\Temp\ipykernel_15332\667503414.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  CAGR = (df["cum_retur